In [450]:
import json
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import string

# process the tweets for sentiment analysis
open the tweets, process them, then save them back into the json format

In [451]:
# open files to read and write from
in_filename = 'SampleData.json'
out_filename = 'prepared_json.json'

in_handle = open(in_filename, encoding='utf8')
out_handle = open(out_filename, 'w', encoding='utf8')

In [452]:
# process the tweet and adds in negation terms
# adds a not to every word after a token of logical negation
# until the next puncutation mark
puncuation = {'"', ',', '.', '?', '!'}

# adapted from https://stackoverflow.com/questions/23384351/how-to-add-tags-to-negated-words-in-strings-that-follow-not-no-and-never
def check_and_negate(tweet: str):
    tweet = tweet.strip()
    # check if the end of the tweet has a puncuation mark and add if not
    if len(tweet) > 0 and tweet[-1] not in puncuation:
        tweet += '.'
    tweet = re.sub('n\'t', ' not', tweet) # replace n't with not
    tweet = re.sub('n’t', ' not', tweet)
    # add NOT_ to the beginning of each word until a end of sentence mark (.,!?) occurs
    tweet = re.sub(r'\b(?:not|never|no)\b[\'\w\s]+[.,?!"]', 
                   lambda match: re.sub(r'(\s+)(\w+)', r'\1NOT_\2', match.group(0)), 
                   tweet,
                   flags=re.IGNORECASE)
    return tweet

In [453]:
# remove stop words
stop_words = set(stopwords.words('english'))

tokenizer = RegexpTokenizer(r'\w+') # remove punctuation and keep only words
ps = PorterStemmer()

def remove_stop_and_stem(tweet: str):
    words = []
    tokens = tokenizer.tokenize(tweet)
    for token in tokens:
        if token not in stop_words:
            words.append(ps.stem(token))
    return ' '.join(words)

In [454]:
# http://www.aclweb.org/anthology/W16-2610

# transform emojis into text
def transform_emojis(tweet: str):
    tweet = re.sub('👏', ' emoji_clap ', tweet)
    tweet = re.sub('😬', ' emoji_nervous ', tweet)
    tweet = re.sub('💯', ' emoji_hundred_points ', tweet)
    tweet = re.sub('🥺', ' emoji_plead ', tweet)
    tweet = re.sub('🤪', ' emoji_silly ', tweet)
    tweet = re.sub('😛', ' emoji_jest ', tweet)
    tweet = re.sub('[😂😭🤣]', ' emoji_joy ', tweet)
    tweet = re.sub('[😢😔]', ' emoji_sad ', tweet)
    tweet = re.sub('😀', ' emoji_happy ', tweet)
    tweet = re.sub('👍', ' emoji_aprove ', tweet)
    tweet = re.sub('😎', ' emoji_cool ', tweet)
    tweet = re.sub('❤️', ' emoji_love ', tweet)
    tweet = re.sub('😆', ' emoji_excitement ', tweet)
    tweet = re.sub('🔥', ' emoji_hot ', tweet)
    return tweet

In [455]:
# processes the tweet
# https://towardsdatascience.com/the-real-world-as-seen-on-twitter-sentiment-analysis-part-one-5ac2d06b63fb
def process_tweet(tweet: str):
    tweet = tweet.lower() # convert to lowercase
    tweet = re.sub(r'#\w*', '', tweet) # remove hashtags
    tweet = re.sub('@\S+', '', tweet) # remove @username
    tweet = re.sub('\s+', ' ', tweet) # remove multiple whitespace
    # https://stackoverflow.com/questions/6038061/regular-expression-to-find-urls-within-a-string
    tweet = re.sub('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', tweet) # remove links
    tweet = transform_emojis(tweet) # transform emojis
    tweet = check_and_negate(tweet)# add negations
    tweet = remove_stop_and_stem(tweet) # remove stop words and stem
    tweet.strip() # remove excess leading and trailing whitespace
    return tweet

In [456]:
j_items = []

for line in in_handle:
    # convert json object into python dict, append to list
    item = json.loads(line)
    j_items.append(item)
    # perform cleaning and preparation
    if 'text' in item: # check that the dictionary contains the key
        print('before:', item['text'])
        item['text'] = process_tweet(item['text']) # convert to lowercase
        print('after:', item['text'])
    # write the json object into an output file
    out_handle.write(json.dumps(item))
    out_handle.write('\n')

before: 👏👏👏  https://t.co/FMzt5rKOZZ
after: clap clap clap
before: @yaminivalli_01 @arrahman https://t.co/SPuWgKsWsL
after: 
before: On today's show...well it's a miracle I made it, since my car is now a block of ice and won't start. Tune in and se… https://t.co/SKa1JA0OIZ
after: today show well miracl made sinc car block ice wo not_start tune se
before: @mags96g Well on top. Great team goal by Sadio.
after: well top great team goal sadio
before: Mi siempre morena de Cardedeu... #teechodemenos #mimorenadeCardedeu #mimorena #tengoganasdeverte #soloconmirarnos… https://t.co/PLdX1yFjtv
after: mi siempr morena de cardedeu
before: Remedy Intelligent Staffing is hiring in #Sanborn, NY! Click the link in our bio to apply: Packager #SupplyChain
after: remedi intellig staf hire ny click link bio appli packag
before: #StayinAlive #HandsOnlyCPR #CPRSavesLives please leave the organ harvesting to the professionals 😬
after: pleas leav organ harvest profession nervou
before: @yxssxnia I know the fee

after: accid clear hwi 99 nb ca 58
before: Voltando do trampo, só espero que a minha ori esteja no freeze esperando 🙃
after: voltando trampo só espero que minha ori esteja freez esperando
before: A very good morning from Al Galayel, Mask of Time and Suburbia, who enjoyed this beautiful sunrise during beach tra… https://t.co/DD10uICx6I
after: good morn al galayel mask time suburbia enjoy beauti sunris beach tra
before: #DirilişSancağı Durcak hiçbir güç yok karşıda
after: durcak hiçbir güç yok karşıda
before: يارب تمم
الكاس يقترب
after: يارب تمم الكاس يقترب
before: Set mixado do @caik22 tá muito foda 🔥🔥
after: set mixado tá muito foda hot hot
before: Maç güzel zaten. Ama sürekli sinek vızıltısı gibi bir ses bütün konsantrasyonu bozuyor. Uğur Meleke denen zararlıyı… https://t.co/6sIcwa0iuX
after: maç güzel zaten ama sürekli sinek vızıltısı gibi bir se bütün konsantrasyonu bozuyor uğur melek denen zararlıyı
before: @LePoint Si le Béninois, M. Adam Pinto que j’ai rencontré le 14 février 201

In [457]:
in_handle.close()
print('input file closed')
out_handle.close()
print('output file closed')

input file closed
output file closed


In [458]:
# Print out a tweet with formatting
print(json.dumps(j_items[1], indent=4))

{
    "created_at": "Wed Jan 30 20:42:09 +0000 2019",
    "id": 1090711786720874500,
    "id_str": "1090711786720874496",
    "text": "",
    "display_text_range": [
        25,
        25
    ],
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "truncated": false,
    "in_reply_to_status_id": 1090711475998515200,
    "in_reply_to_status_id_str": "1090711475998515201",
    "in_reply_to_user_id": 4894106887,
    "in_reply_to_user_id_str": "4894106887",
    "in_reply_to_screen_name": "yaminivalli_01",
    "user": {
        "id": 297542334,
        "id_str": "297542334",
        "name": "\u0430\u0434\u0438\u0442\u0439\u0430",
        "screen_name": "Backstreet_Boy3",
        "location": "Madras",
        "url": "https://www.facebook.com/SuperstarPrinceMaheshBabuGhattamaneni",
        "description": "Hated by many, wanted by plenty, disliked by some, confronted by none! \ud83c\udfc7",
        "translator_type": "none",
        "pr

In [459]:
print(json.dumps(j_items[2], indent=4))

{
    "created_at": "Wed Jan 30 20:42:08 +0000 2019",
    "id": 1090711785651474400,
    "id_str": "1090711785651474432",
    "text": "today show well miracl made sinc car block ice wo not_start tune se",
    "display_text_range": [
        0,
        140
    ],
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "truncated": true,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 237010321,
        "id_str": "237010321",
        "name": "Andy Jorgensen",
        "screen_name": "StuckWithAndy",
        "location": "Milton, WI",
        "url": "https://www.facebook.com/repjorgensen",
        "description": "Former democratic leader in the Wisconsin State Assembly, now has his own radio show Stuck In The Middle with Andy Jorgensen on 940am WFAW from 3-5pm!",
        "transla

In [460]:
# test the negation
test = "really? sure hope this isn’t right because it just wrong. good grief. respectfully submitted."
print(process_tweet(test))

realli sure hope not_right not_becaus not_it not_just not_wrong good grief respect submit
